In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime

In [69]:
DATASET_SIZE = 480
DATASET_IS_BALANCED = False


MAX_FEATURES = 20000        # max_features params for CountVectorizer

training_name = 'glove-cnn-{}_{}k_{}'.format(
    MAX_FEATURES,
    DATASET_SIZE,
    'bal' if DATASET_IS_BALANCED else 'imbal'
)

training_args_datetime = datetime(year=2023, month=12, day=20)
training_storing_folder = Path(training_name).resolve()

In [70]:
import pickle

import tensorflow as tf
import keras

# load the tf model
# either a end-to-end
# or build our own (by loading the vectorizer and the model)


text_vectorizer_path = Path.joinpath(training_storing_folder, "{}_{}_textvectorizer.pkl".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
))
vectorizer_from_disk = pickle.load(open(text_vectorizer_path, 'rb'))
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=MAX_FEATURES,
    output_sequence_length=512)

vectorizer.set_weights(vectorizer_from_disk['weights'])

model_path = Path.joinpath(training_storing_folder, "{}_{}_model.keras".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
))
model = keras.models.load_model(model_path)

end_to_end_model_path = Path.joinpath(training_storing_folder, "{}_{}_end2end.keras".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
))

end_to_end_model = keras.models.load_model(end_to_end_model_path)

print('\n\n')
print('Loaded text vectorizer from {}'.format(text_vectorizer_path))
print('Loaded model from {}'.format(model_path))
print('Loaded end to end model from {}'.format(end_to_end_model_path))




Loaded text vectorizer from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_imbal/glove-cnn-20000_480k_imbal_2023-12-20_textvectorizer.pkl
Loaded model from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_imbal/glove-cnn-20000_480k_imbal_2023-12-20_model.keras
Loaded end to end model from /root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_imbal/glove-cnn-20000_480k_imbal_2023-12-20_end2end.keras


In [71]:
end_to_end_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 512)               0         
 ctorization)                                                    
                                                                 
 model (Functional)          (None, 2)                 6808154   
                                                                 
Total params: 6808154 (25.97 MB)
Trainable params: 6808154 (25.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
vectorizer.get_vocabulary()[:20]

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'to',
 'game',
 'i',
 'it',
 'you',
 'of',
 'is',
 'this',
 'in',
 'that',
 'for',
 'but',
 'with',
 's',
 'on']

In [73]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 512, 300)             6000600   ['input_1[0][0]']             
                                                                                                  
 conv1d (Conv1D)             (None, 506, 128)             268928    ['embedding[0][0]']           
                                                                                                  
 conv1d_1 (Conv1D)           (None, 506, 128)             268928    ['embedding[0][0]']           
                                                                                              

In [74]:
# save the .keras model to SavedModel format
# which include a .pb file

# https://www.tensorflow.org/tutorials/keras/save_and_load#savedmodel_format

# model.save(Path.joinpath(training_storing_folder, "{}_{}_model_savedmodel".format(
#     training_name,
#     training_args_datetime.strftime("%Y-%m-%d")
# )))

In [75]:
# save the pretrained model to onnx

import tf2onnx
import onnx

onnx_model_path = Path.joinpath(training_storing_folder, "{}_{}_modelonly.onnx".format(
    training_name,
    training_args_datetime.strftime("%Y-%m-%d")
))

print(training_storing_folder)
print('\n\n')
print(onnx_model_path)

# Keras models and tf functions and can be converted directly within python
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=[tf.TensorSpec([None,512], dtype=tf.int32, name='input')],     # match the input name of the model
    # extra_opset=['ai.onnx.contrib:1'],
    opset=15        # default opset of tf2onnx for testing
)

onnx.save(onnx_model, onnx_model_path)

/root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_imbal



/root/FYP/NLP/dev-workspace/sa/glove-cnn_2023-12-12/glove-cnn-20000_480k_imbal/glove-cnn-20000_480k_imbal_2023-12-20_modelonly.onnx
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'


Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'NoneType' object has no attribute 'name'


2023-12-28 12:23:47.226610: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-28 12:23:47.226659: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-12-28 12:23:47.226738: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2023-12-28 12:23:47.226977: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-28 12:23:47.227006: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-28 12:23:47.227019: I external/local_xla/xl

In [76]:
# https://github.com/onnx/tensorflow-onnx/issues/1867
# "probably there's no shared-name keyword for hash tables in TextVectorization layer."
# Therefore we only convert the model without the vectorizer (resulting in like huggingface, that the tokenizer is not included in onnx)

# !python -m tf2onnx.convert --saved-model "glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_model_savedmodel" --output "glove-cnn-20000_120k_bal/glove-cnn-20000_120k_bal_2023-12-22_modelonly.onnx" --extra_opset ai.onnx.contrib:1 --opset 15

In [77]:
import onnxruntime as rt

sess = rt.InferenceSession(
    onnx_model_path,
    providers=['CPUExecutionProvider']
)

input_name = [inp.name for inp in sess.get_inputs()][0]     # only one input in this model
label_names = [label.name for label in sess.get_outputs()]  # it outputs the label and the probability

In [78]:
test_data = [['I like the game'], ["I do not like it."], ["It crashes when I just run on my pc."]]

In [79]:
# test inference

pred_keras = []
perd_onnx = []

for i in range(len(test_data)):
    # keras inference
    pred_keras.append(end_to_end_model.predict(test_data[i]))

    # onnx inference
    v_out = vectorizer(test_data[i])
    perd_onnx.append(sess.run(None, {"input": v_out.cpu().numpy().astype(np.int32)}))

print(pred_keras)
print(perd_onnx)

for i in range(len(test_data)):
    result_keras = pred_keras[i]
    result_onnx = perd_onnx[i][0]

    np.testing.assert_allclose(result_keras, result_onnx, rtol=1e-3, atol=1e-3)

1/1 [==============================] - 0s 17ms/step
[array([[0.11797059, 0.8820294 ]], dtype=float32), array([[0.64781755, 0.35218248]], dtype=float32), array([[0.7450074, 0.2549926]], dtype=float32)]
[[array([[0.11796283, 0.88203716]], dtype=float32)], [array([[0.647815, 0.352185]], dtype=float32)], [array([[0.74496186, 0.25503814]], dtype=float32)]]
